# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,RENT,26000.0,Source Verified,Jan-2019,low_risk,n,9.60,...,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0,N,N
68813,12000.0,0.2727,368.37,RENT,63000.0,Not Verified,Jan-2019,low_risk,n,29.07,...,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0,N,N
68814,5000.0,0.1992,185.62,MORTGAGE,52000.0,Source Verified,Jan-2019,low_risk,n,14.86,...,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0,N,N
68815,40000.0,0.0646,1225.24,MORTGAGE,520000.0,Verified,Jan-2019,low_risk,n,9.96,...,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0,N,N


## Split the Data into Training and Testing

In [5]:
label_encoder = LabelEncoder()
label_encoder.fit(df['issue_d'])
list(label_encoder.classes_)

['Feb-2019', 'Jan-2019', 'Mar-2019']

In [6]:
df["month_le"] = label_encoder.transform(df["issue_d"])
df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,month_le
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N,2
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N,2
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N,2
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N,2
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,RENT,26000.0,Source Verified,Jan-2019,low_risk,n,9.60,...,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0,N,N,1
68813,12000.0,0.2727,368.37,RENT,63000.0,Not Verified,Jan-2019,low_risk,n,29.07,...,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0,N,N,1
68814,5000.0,0.1992,185.62,MORTGAGE,52000.0,Source Verified,Jan-2019,low_risk,n,14.86,...,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0,N,N,1
68815,40000.0,0.0646,1225.24,MORTGAGE,520000.0,Verified,Jan-2019,low_risk,n,9.96,...,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0,N,N,1


In [7]:
months_num = {
    'Jan-2019' : 1,
    'Feb-2019' : 2,
    'Mar-2019' : 3,
    'Apr-2019' : 4,
    'May-2019' : 5,
    
}

df["month_num"] = df["issue_d"].apply(lambda x: months_num[x])
df["nxt_pymnt_dt"] = df["next_pymnt_d"].apply(lambda x: months_num[x])
#drop all the issue_d, next next_pymnt_d, month_le
#then look at getting the last columns
# used the get_dummies to fill in the boolean values. yes no or 1, 0 
cleaned_df = pd.get_dummies(df, columns =['home_ownership', #rent , mort, own 
                                          #'hardship_flag', 
                                          #'debt_settlement_flag', 
                                          'application_type'], # joint 0 single 1
                                          drop_first= True)
cleaned_df

,loan_amnt,int_rate,installment,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,...,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,month_le,month_num,nxt_pymnt_dt,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,0.0,...,61987.0,N,N,2,3,5,0,0,1,0
1,25000.0,0.2000,929.09,105000.0,Verified,Mar-2019,low_risk,n,20.23,0.0,...,49197.0,N,N,2,3,5,1,0,0,0
2,20000.0,0.2000,529.88,56000.0,Verified,Mar-2019,low_risk,n,24.26,0.0,...,43144.0,N,N,2,3,5,1,0,0,0
3,10000.0,0.1640,353.55,92000.0,Verified,Mar-2019,low_risk,n,31.44,0.0,...,76506.0,N,N,2,3,5,0,0,1,0
4,22000.0,0.1474,520.39,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,0.0,...,20000.0,N,N,2,3,5,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,Source Verified,Jan-2019,low_risk,n,9.60,0.0,...,5425.0,N,N,1,1,5,0,0,1,0
68813,12000.0,0.2727,368.37,63000.0,Not Verified,Jan-2019,low_risk,n,29.07,0.0,...,62939.0,N,N,1,1,5,0,0,1,0
68814,5000.0,0.1992,185.62,52000.0,Source Verified,Jan-2019,low_risk,n,14.86,0.0,...,18492.0,N,N,1,1,5,1,0,0,0
68815,40000.0,0.0646,1225.24,520000.0,Verified,Jan-2019,low_risk,n,9.96,0.0,...,78634.0,N,N,1,1,5,1,0,0,0


In [8]:
# Create our features 
## encode the app type. 
## keep the loan status as the 
X = x_cols = [i for i in cleaned_df.columns if i not in ('loan_status',
                                                 'pymnt_plan', 
                                                 'hardship_flag',
                                                 'debt_settlement_flag',
                                                 'verification_status',
                                                 'initial_list_status',
                                                        'issue_d',
                                                        'next_pymnt_d',
                                                        'month_le')]
X = cleaned_df[x_cols]

# Create our target
y = cleaned_df['loan_status']



In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 82 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   68817 non-null  float64
 1   int_rate                    68817 non-null  float64
 2   installment                 68817 non-null  float64
 3   annual_inc                  68817 non-null  float64
 4   dti                         68817 non-null  float64
 5   delinq_2yrs                 68817 non-null  float64
 6   inq_last_6mths              68817 non-null  float64
 7   open_acc                    68817 non-null  float64
 8   pub_rec                     68817 non-null  float64
 9   revol_bal                   68817 non-null  float64
 10  total_acc                   68817 non-null  float64
 11  out_prncp                   68817 non-null  float64
 12  out_prncp_inv               68817 non-null  float64
 13  total_pymnt                 688

In [10]:
# Check the balance of our target values
y = df["loan_status"].values.reshape(-1, 1)
y

array([['low_risk'],
       ['low_risk'],
       ['low_risk'],
       ...,
       ['low_risk'],
       ['low_risk'],
       ['low_risk']], dtype=object)

In [11]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,month_num,nxt_pymnt_dt,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,application_type_Joint App
39692,10000.0,0.1131,328.86,49000.0,24.54,0.0,0.0,10.0,1.0,6580.0,...,309140.0,56262.0,9500.0,54197.0,1,5,1,0,0,0
37830,21000.0,0.1072,684.74,57400.0,34.52,0.0,0.0,12.0,0.0,18688.0,...,98100.0,35707.0,64700.0,30000.0,1,5,0,0,1,0
64531,35000.0,0.0881,1109.90,180000.0,3.01,0.0,2.0,7.0,0.0,836.0,...,327877.0,1222.0,41500.0,26177.0,1,5,0,1,0,0
28730,20000.0,0.0819,628.49,80000.0,26.51,0.0,0.0,15.0,0.0,2615.0,...,143541.0,81162.0,14500.0,111641.0,2,5,0,1,0,0
35102,12600.0,0.1356,427.96,71000.0,22.38,1.0,2.0,7.0,0.0,9073.0,...,316682.0,33433.0,26500.0,50182.0,2,5,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18872,35000.0,0.0881,1109.90,400000.0,13.79,0.0,1.0,17.0,1.0,7810.0,...,631699.0,189478.0,28600.0,244399.0,2,4,1,0,0,0
6415,16800.0,0.0881,532.76,31000.0,20.48,0.0,0.0,3.0,0.0,1867.0,...,20400.0,11771.0,3500.0,16900.0,3,5,0,1,0,0
12632,25000.0,0.1102,818.71,90000.0,15.27,1.0,0.0,18.0,0.0,26106.0,...,195681.0,40625.0,40600.0,33181.0,2,4,1,0,0,0
24022,14000.0,0.1072,456.49,180000.0,12.35,2.0,0.0,10.0,0.0,3601.0,...,754884.0,49607.0,11000.0,63655.0,2,5,1,0,0,0


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [12]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [13]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [14]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [15]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=1)

In [16]:
y_pred_rf = brf.predict(X_test_scaled)

In [17]:
# Cafrom sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred_rf)

0.7782743311134478

In [18]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_rf)

array([[   58,    29],
       [ 1885, 15233]])

In [19]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.67      0.89      0.06      0.77      0.58        87
   low_risk       1.00      0.89      0.67      0.94      0.77      0.61     17118

avg / total       0.99      0.89      0.67      0.94      0.77      0.61     17205



In [27]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
sorted(zip(brf.feature_importances_, X.columns), reverse=True)
## visualize to make it look pretty for eveyone to read

## office hours.. go over if we shoud include the important features. due to the data set being so large in size.
# with all the columns its go big to vizualize
#importances_df = pd.DataFrame(sorted(zip(brf.feature_importances_, X.columns), reverse=True))
#importances_df.set_index(importances_df[1], inplace=True)
#importances_df.drop(columns=1, inplace=True)
#importances_df.rename(columns={0: 'Feature Importances'}, inplace=True)
#importances_sorted = importances_df.sort_values(by='Feature Importances')
#importances_sorted.plot(kind='barh', color='lightgreen', title= 'Features Importances', legend=False)

[(0.07065199779595237, 'total_rec_prncp'),
 (0.06713294588827896, 'last_pymnt_amnt'),
 (0.06270180730837073, 'total_pymnt'),
 (0.056361971713964315, 'total_rec_int'),
 (0.050975725307769705, 'total_pymnt_inv'),
 (0.029643734778720336, 'int_rate'),
 (0.02924799074286322, 'month_num'),
 (0.017859532827436877, 'avg_cur_bal'),
 (0.01779385709577484, 'max_bal_bc'),
 (0.01772348146690332, 'annual_inc'),
 (0.01761495003477694, 'dti'),
 (0.01707202383398014, 'out_prncp_inv'),
 (0.017046329198688266, 'installment'),
 (0.016856418006393675, 'mo_sin_old_rev_tl_op'),
 (0.01630558048669328, 'out_prncp'),
 (0.016299214222929296, 'all_util'),
 (0.015972843501797684, 'bc_util'),
 (0.015119685726850298, 'mths_since_recent_inq'),
 (0.014433860758518921, 'mo_sin_old_il_acct'),
 (0.01437013329281371, 'total_bal_il'),
 (0.014251274587225478, 'il_util'),
 (0.01411230366944374, 'tot_hi_cred_lim'),
 (0.014026702600889271, 'mths_since_rcnt_il'),
 (0.013938957012199263, 'total_il_high_credit_limit'),
 (0.013906

### Easy Ensemble Classifier

In [21]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [22]:
# Calculated the balanced accuracy score
Predictions = eec.predict(X_test_scaled)
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, Predictions)

0.9316126199080621

In [23]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, Predictions)

array([[   80,     7],
       [  964, 16154]])

In [24]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, Predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.92      0.94      0.14      0.93      0.87        87
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17118

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



### Final Questions

1. Which model had the best balanced accuracy score?

## Easy Ensemble Classifier
It had a accuracy score of _.93_ vs the _.77_

2. Which model had the best recall score?

## Easy Ensemble Classifier
the _Recall_  is is _.92_ vs _.67_ is much higher then the _RFC_.

3. Which model had the best geometric mean score?

## Easy Ensemble Classifier
the _Geomean_ is _.93_ vs the _RFC mean_ of _.77_

4. What are the top three features?

-total_rec_prncp

-last_pymnt_amnt

-total_pymnt

